In [1]:
import kwikfile as kf
import numpy as np
import imp
%matplotlib inline
import matplotlib.pyplot as plt
from klusta.kwik import KwikModel
#from phy.io import KwikModel
import matplotlib as mpl
from pylab import *

import ipywidgets as widgets
from IPython.display import display
import time

class ColorMap:
    """!  @brief model that returns an average detection of each site of a given kwik file.
    
    @author: Harison P B de Oliveira
    @date: 2018.May.23
    """
    
    def __init__(self,path=None):
        if path is None:
            print ("It still with no path")
        else:
            self.kwik_model=[]
            self.kwikfile=[]
            for i in range(0,6):
                kpath = path+'/0'+str(i+1)+'/g0'+str(i+1)+'.kwik'
                self.kwik_model.append(KwikModel(kpath));
                self.kwikfile.append(kf.KwikFile(kpath))
            #print(self.kwikfile[0])
            #self.kwik_model=KwikModel(kpath)
            #self.kwikfile=kf.KwikFile(kpath)
            #self.kpath=kpath
            #if name is None:
            #    self.name=self.kwikfile.name
            #else:
            #    self.name=name
            #print ("Created class on = %s !" % self.kpath)
            
    def get_median_amplitude_group(self):
        """!  @brief Returns the average amplitude of all sites of the non-noisy groups.
    
        @author: Harison P B de Oliveira
        @date: 2018.Jun.07
        """
        
        a=np.zeros((6,10))
        for shank in range(0,6):
            spk_id=self.kwikfile[shank].all_spike_id_on_groups(group_names=
                                                               self.kwikfile[shank].list_of_non_noisy_groups())
            w=self.kwik_model[shank].all_waveforms
            wf=w[spk_id]
            #wf=w[spk_id[0:10000]]
            
            median_amplitudes=dict()
            
            print("Shank 0"+str(shank+1)+":")
            print("Loading the average amplitude for each site.")
            
            amplitudes=[]
            A=dict()
            
            progress = widgets.IntProgress(min=0, description='Loading channels:'
                                       ,max=wf.shape[2])#add kwikfile
            display(progress)
            
            for elect in range(0,wf.shape[2]):#add kwikfile
                A[elect]=[]
                for i in range(0,len(wf)):
                    wave=wf[i,:,elect]
                    A[elect].append(np.max(wave)-np.min(wave))
                amplitudes.append(np.mean(A[elect]))
                progress.value += 1
            a[shank,:]=amplitudes
            """
        a=np.zeros((6,10))
        for i in range(0,6):
            a[i,:]=amplitudes
        """
        return a

        
    def plot_median_amplitude(self,amplitudes=None):
        """!  @brief Returns an image with maximum average amplitude of each site of a shank.
    
        @author: Harison P B de Oliveira
        @date: 2018.Jun.07
        """
        
        if amplitudes is None:
            return None
        else:
            fig,ax = plt.subplots(figsize=(60, 20))
            num_channels=len(amplitudes)

            posx=np.flipud (self.kwik_model[0].channel_positions [:,0])
            posy=np.flipud (self.kwik_model[0].channel_positions [:,1])
            #print(posx,posy)
            
            a=list()
            
            cmap = mpl.cm.jet
            for shank in range(0,6):
                norm = mpl.colors.Normalize(vmin=np.min(amplitudes[shank,:]), vmax=np.max(amplitudes[shank,:]))
                cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                        norm=norm,
                                        extend='both',
                                        extendfrac='auto',
                                        spacing='uniform',
                                        orientation='vertical') 
                dx=10
                offset=shank*200
                for ch in range (0,10):
                    if ch%2==0:
                        x_offset=posx[ch]+dx+offset
                    else:
                        x_offset=posx[ch]-dx+offset
                        dx+=10
                    #x_offset = posx[ch]
                    y_offset =posy[ch]*2
                    circle = plt.Circle((x_offset,y_offset),10,color=cb1.to_rgba(amplitudes[shank,ch]))
                    a.append(circle)
            
            ax.cla()
            ax.set_xlim((min(posx)-dx-10, max(posx)+dx+1100))
            ax.set_ylim((min(posy)*2-10, max(posy)*2+10))
            for i in range(0,len(a)):
                c=a[i]
                ax.add_artist(c)
            
            data = np.clip(np.randn(0, 0), -1, 1)
            cax = ax.imshow(data, interpolation='None', cmap=mpl.cm.jet)
            cbar = fig.colorbar(cax,ticks=[0,0.5,1],orientation='vertical')
            cbar.ax.set_yticklabels(['Low','Medium','High'])
            plt.gca().invert_yaxis()
            plt.axis('off')

            plt.show()
            
            
        

/home/harison/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/harison/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
path="/home/harison/UFPE/data"
cm = ColorMap(path=path)

OSError: File `/home/harison/UFPE/data/01/g01.kwik` doesn't exist.

In [ ]:
import time
start_time = time.time()

a=cm.get_median_amplitude_group()

elapsed_time = time.time() - start_time
time.strftime("%M:%S", time.gmtime(elapsed_time))

In [ ]:
cm.plot_median_amplitude(amplitudes=a)

In [2]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text
import ipywidgets as widgets

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center'
)

data = [
    Box([Label(value='Path'),
            Text("/home/harison/UFPE/data/g01.kwik")
            ], layout=form_item_layout),
    Box([Label(value='Group name:'),
         Dropdown(options=['good', 'mua', 'noise'])],layout=form_item_layout)
    
]

form = Box(data, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
display(form)

button = widgets.Button(description="Enviar informações!")
display(button)

def on_button_clicked(b):
    #kpath=form_items[0].children[0].value
    kpath=data[0].children[1].value
    group_name=data[1].children[1].value
    hm = ColorMap(kpath)
    ampl=hm.group_amplitude(group_name)
    #hm.amplitudes_heat_map(ampl)
    #group_name=
    #print("%s: %s" %(children[0].description,children[0].value))
    
    
button.on_click(on_button_clicked)

Box(children=(Box(children=(Label(value='Path'), Text(value='/home/harison/UFPE/data/g01.kwik')), layout=Layout(display='flex', flex_flow='row', justify_content='center')), Box(children=(Label(value='Group name:'), Dropdown(options=('good', 'mua', 'noise'), value='good')), layout=Layout(display='flex', flex_flow='row', justify_content='center'))), layout=Layout(align_items='stretch', border='solid 2px', display='flex', flex_flow='column', width='50%'))

Button(description='Enviar informações!', style=ButtonStyle())